In [1]:
# langgraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# langchain
from langchain_groq import ChatGroq

# pydantic
from pydantic import BaseModel

# others
from typing import Annotated
import random
import gradio as gr
from IPython.display import Image, display

import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

/home/rmede/projects/agents-course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Criando o estado

In [2]:
class State(BaseModel):
    messages: Annotated[list, add_messages]

# Iniciando o construtor de grafo

In [3]:
graph_builder = StateGraph(State)

# Criando nó

In [5]:
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0,
    max_tokens=300
)

In [6]:
def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot_node", chatbot_node)

# Criando edges

In [7]:
graph_builder.add_edge(START, "chatbot_node")
graph_builder.add_edge("chatbot_node", END)

# Compilando o grafo

In [8]:
graph = graph_builder.compile()

# Rodando tudo

In [ ]:
# Criando função apenas para rodar no gradio
def chat(user_input: str):
    initial_state = State(
        messages=[
            {
                "role": "user",
                "content": user_input
            }
        ]
    )

    result = graph.invoke(initial_state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


{'messages': [HumanMessage(content='Olá, tudo bem?', additional_kwargs={}, response_metadata={}, id='b9f6f2b3-c36d-4fc8-a6f5-edb1361893d0'), AIMessage(content='Tudo bem, sim! É um prazer conversar com você! Como posso ajudar hoje?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 16, 'total_tokens': 37, 'completion_time': 0.049630323, 'prompt_time': 0.002563934, 'queue_time': 0.268819944, 'total_time': 0.052194257}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_42ae451038', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--396f8cce-5567-4039-bb03-8d480d518274-0', usage_metadata={'input_tokens': 16, 'output_tokens': 21, 'total_tokens': 37})]}
{'messages': [HumanMessage(content='Gostaria de saber quem foi o campeão da NBA em 2000', additional_kwargs={}, response_metadata={}, id='8e312767-ad8e-451a-bc23-2307374aa88c'), AIMessage(content="O campeão da NBA em 2000 foi o 